# Set up

In [1]:
%pip install rdflib
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from rdflib import Graph, URIRef
import os

data_folder = "Data"
g = Graph()
g.parse("Data/rdf_400_sphn.txt", format="turtle")
#g.parse("Data/rdf_300_sphn.txt", format="turtle")
#g.parse("Data/rdf_400_sphn.txt", format="turtle")
print(f"Loaded {len(g)} triples.")

Loaded 182948 triples.


## Checking the graph

In [35]:
predicates = set()
for _, p, _ in g.triples((None, None, None)):
    predicates.add(p)

for predicate in predicates:
    print(predicate)

https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug


In [36]:
subject_types = set()
for s, p, o in g.triples((None, None, None)):
    if p == URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'):
        subject_types.add(o)

for t in subject_types:
    print(t)

https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription
https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis
https://www.biomedit.ch/rdf/sphn-schema/sphn/Drug
https://www.biomedit.ch/rdf/sphn-schema/sphn/LabTestEvent
https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult
https://www.biomedit.ch/rdf/sphn-schema/sphn/Substance
https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier
https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure


In [37]:
unique_predicates = set()
for _, predicate, _ in g.triples((None, None, None)):
    unique_predicates.add(predicate)

for pred in unique_predicates:
    print(pred)

https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient
https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug


In [38]:
object_types = set()
for _, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), None)):
    object_types.add(o)

for obj_type in object_types:
    print(obj_type)

https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription
https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis
https://www.biomedit.ch/rdf/sphn-schema/sphn/Drug
https://www.biomedit.ch/rdf/sphn-schema/sphn/LabTestEvent
https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult
https://www.biomedit.ch/rdf/sphn-schema/sphn/Substance
https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier
https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure


In [85]:
example_tuples = []
predicate_uri = URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion')
for triple in g.triples((None, predicate_uri, None)):
    example_tuples.append(triple)
    if len(example_tuples) >= 15:
        break

for tup in example_tuples:
    print(tup)

(rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis/0/patients/14072553'), rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion'), rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/v10'))
(rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis/0/patients/12429250'), rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion'), rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/v10'))
(rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis/0/patients/19261730'), rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion'), rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/v10'))
(rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure/1/patients/17859045'), rdflib.term.URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion'), rdfli

# Star Shape Queries

## 1 Hop

In [40]:

qa_pairs = []
for s, p, o in g.triples((None, None, None)):
    if 'PATIENTS' in str(s):
        patient_id = str(s).split('/')[-1]
        if patient_id.isdigit():
            question = f"What medical procedures and diagnoses did patient {patient_id} have"
            # SPARQL query template (adjust predicates as needed)
            sparql_query = f"""
            SELECT ?MedicalProcedure ?Diagnosis WHERE {{
                ?MedicalProcedure <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{s}> .
                ?MedicalProcedure a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .
                ?Diagnosis <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{s}> .
                ?Diagnosis a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
            }}
            """
            qa_pairs.append((question, sparql_query.strip()))

#for q, a in qa_pairs:
#    print("Q:", q)
#    print("SPARQL:", a)
#    print()
len(qa_pairs)

100

In [41]:

#print(qa_pairs[1][1])
for row in g.query(qa_pairs[10][1]):
    print([str(item) for item in row])

## 2 Hop

In [42]:
# Generate question and SPARQL query pairs for the new question template
new_qa_pairs = []
for s, p, o in g.triples((None, None, None)):
    if 'PATIENTS' in str(o): 
        if (str(s).split('/')[-1]).isdigit():
            #print (o)
            patient_id = str(o).split('/')[-1]
            subject = "https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/" + patient_id
            question = f"What was the code of a test recorded for patient {patient_id}, who also had a Drug Prescription?"
            sparql_query = f"""
             SELECT ?Code WHERE {{
                ?DrugPrescription <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{subject}> .
                ?LabTestEvent <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{subject}> .
                ?LabTestEvent <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult .
                ?LabResult <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?Code .
                
             }}
             """
            new_qa_pairs.append((question, sparql_query.strip()))
len(new_qa_pairs)

7929

In [43]:
print(new_qa_pairs[1][1])
for row in g.query(new_qa_pairs[1][1]):
    print([str(item) for item in row])

SELECT ?Code WHERE {
                ?DrugPrescription <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/19164956> .
                ?LabTestEvent <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/19164956> .
                ?LabTestEvent <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult .
                ?LabResult <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?Code .

             }
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/51146']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/51146']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/51146']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/51146']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/51146']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/51146']
['https://www.biomedit.ch/rdf/

## 2 Hop Chain

In [44]:
active_ingredient_qa_pairs = []
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription'))):
    prescription_id = str(s).split('/')[-1]
    question = f"What active ingredients are in drug prescription {prescription_id}?"
    sparql_query = f"""
    SELECT ?ActiveIngredient WHERE {{
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
        ?Drug <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient> ?ActiveIngredient .
    }}
    """
    active_ingredient_qa_pairs.append((question, sparql_query.strip()))

if active_ingredient_qa_pairs:
    print(active_ingredient_qa_pairs[0][0])
    print(active_ingredient_qa_pairs[0][1])
    print(len(active_ingredient_qa_pairs))

What active ingredients are in drug prescription 19347414?
SELECT ?ActiveIngredient WHERE {
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription/3/patients/19347414> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
        ?Drug <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient> ?ActiveIngredient .
    }
2798


In [45]:
for row in g.query(active_ingredient_qa_pairs[5][1]):
    print([str(item) for item in row])

['https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription/12/patients/15065147/Substance']


# Medium Comp Qns

## 1 Hop Star

In [46]:
procedure_qa_pairs = []
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure'))):
    procedure_id = str(s).split('/')[-1]
    question = f"What patients, Code and Coding System Version does medical procedure {procedure_id} have?"
    sparql_query = f"""
    SELECT ?Subject ?Code ?CodingSystemVersion WHERE {{
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Subject .
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?Code .
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion> ?CodingSystemVersion .
    }}
    """
    procedure_qa_pairs.append((question, sparql_query.strip()))

print(len(procedure_qa_pairs))
if procedure_qa_pairs:
    print(procedure_qa_pairs[0][0])
    print(procedure_qa_pairs[0][1])

164
What patients, Code and Coding System Version does medical procedure 19303101 have?
SELECT ?Subject ?Code ?CodingSystemVersion WHERE {
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure/3/patients/19303101> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Subject .
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure/3/patients/19303101> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?Code .
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure/3/patients/19303101> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion> ?CodingSystemVersion .
    }


In [47]:
for row in g.query(procedure_qa_pairs[0][1]):
    print([str(item) for item in row])

['https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/19303101', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/icd#03BC4ZZ', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/v10']


## 3 Hop Chain

In [48]:
substance_code_qa_pairs = []
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription'))):
    prescription_id = str(s).split('/')[-1]
    question = f"What are the codes for the substances used in the drug prescription {prescription_id}?"
    sparql_query = f"""
    SELECT ?SubstanceCode WHERE {{
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
        ?Drug <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient> ?Substance .
        ?Substance <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?SubstanceCode .
    }}
    """
    substance_code_qa_pairs.append((question, sparql_query.strip()))

print(len(substance_code_qa_pairs))
if substance_code_qa_pairs:
    print(substance_code_qa_pairs[0][0])
    print(substance_code_qa_pairs[0][1])

2798
What are the codes for the substances used in the drug prescription 19347414?
SELECT ?SubstanceCode WHERE {
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription/3/patients/19347414> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
        ?Drug <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient> ?Substance .
        ?Substance <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?SubstanceCode .
    }


In [49]:
for row in g.query(substance_code_qa_pairs[3][1]):
    print([str(item) for item in row])

['https://www.biomedit.ch/rdf/sphn-schema/sphn/ndc#378580793']


## 1 Hop Chain

In [50]:
drug_qa_pairs = []
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription'))):
    prescription_id = str(s).split('/')[-1]
    question = f"What are the drugs in drug prescription {prescription_id}?"
    sparql_query = f"""
    SELECT ?Drug WHERE {{
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
    }}
    """
    drug_qa_pairs.append((question, sparql_query.strip()))

print(len(drug_qa_pairs))
if drug_qa_pairs:
    print(drug_qa_pairs[0][0])
    print(drug_qa_pairs[0][1])

2798
What are the drugs in drug prescription 19347414?
SELECT ?Drug WHERE {
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription/3/patients/19347414> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
    }


In [51]:
for row in g.query(drug_qa_pairs[3][1]):
    print([str(item) for item in row])

['https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription/1/patients/18533998/Drug']


# Chat GPT Generation

In [60]:
%pip install openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [95]:
from openai import OpenAI
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv(dotenv_path="Data/.env")
client = OpenAI()

# Read prompt from file
with open("Data/prompt.txt", "r") as file:
    prompt_text = file.read()

# Read graph text data
with open("Data/ontology.ttl", "r") as f:
    graph_data = f.read()

# Combine both into a full prompt
full_prompt = f"{prompt_text}\n\n{graph_data}"


# Query OpenAI
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": full_prompt}
    ]
)

generated_response = response.choices[0].message.content

# Save to file
with open("Data/output.txt", "w", encoding="utf-8") as out_file:
    out_file.write(generated_response)

print("\n✅ Response saved to output.txt")


✅ Response saved to output.txt


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="Data/.env")
client = OpenAI()

try:
    models = client.models.list()
    print("✅ API key is valid. Available models:")
    for m in models.data:
        print(m.id)
except Exception as e:
    print("❌ Error:", e)

In [88]:
output_file = "Data/output.txt"

if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as file:
        content = file.read()

    # Replace all instances of X or Y with {X} and {Y}
    updated_content = content.replace("{X}", "X").replace("{Y}", "Y")

    # Save the updated content back to the file
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(updated_content)

    print("✅ Updated content saved to output.txt")
else:
    print("❌ File not found.")

✅ Updated content saved to output.txt


In [127]:
import os

output_file = "Data/output.txt"

question_query_pairs = []

if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            if line.strip():  # Skip empty lines
                updated_line = " ".join(line.split()[1:])
                question_query_pairs.append(updated_line)
list1 = question_query_pairs[::2]  # Elements at even indices
list2 = question_query_pairs[1::2]  # Elements at odd indices

# Combine the two lists into one list of tuples
combined_list = list(zip(list1, list2))

print(f"Loaded {len(combined_list)} question-query pairs.")

Loaded 100 question-query pairs.


In [ ]:
updated_question_query_pairs = []

for question, query in combined_list:
    # Remove the first token from the question
    updated_question = " ".join(question.split()[1:])
    updated_question_query_pairs.append((updated_question, query))

# Print the updated pairs
for q, a in updated_question_query_pairs[:20]:  # Display the first 10 updated pairs
    print("Updated Question:", q)
    print("SPARQL Query:", a)

In [128]:
import yaml

output_yaml = "question_query_templates.yaml"

# Consolidating data
data_to_write = [{"Question": q, "Query": a} for q, a in combined_list]

with open(output_yaml, mode="w", encoding="utf-8") as file:
    yaml.dump(data_to_write, file, default_flow_style=False, allow_unicode=True)

print(f"YAML file '{output_yaml}' created successfully.")

YAML file 'question_query_templates.yaml' created successfully.


# Generating qqpairs from the templates

In [2]:
import yaml

# Load data from the YAML file
yaml_file = "question_query_templates.yaml"

with open(yaml_file, mode="r", encoding="utf-8") as file:
    question_query_data = yaml.safe_load(file)
len(question_query_data)

100

In [ ]:
#Load all instances of classes in individual lists
patient_instances = set()
diagnosis_instances = set()
drug_instances = set()
medical_procedure_instances = set()
drug_prescription_instances = set()
lab_test_event_instances = set()
lab_result_instances = set()
substance_instances = set()

for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier'))):
    patient_instances.add(str(s))
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis'))):
    diagnosis_instances.add(str(s))
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/Drug'))):
    drug_instances.add(str(s))
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure'))):
    medical_procedure_instances.add(str(s))
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription'))):
    drug_prescription_instances.add(str(s))
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/LabTestEvent'))):
    lab_test_event_instances.add(str(s))
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult'))):
    lab_result_instances.add(str(s))
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/Substance'))):
    substance_instances.add(str(s))

# Print the number of instances found
print(f"Number of patient instances: {len(patient_instances)}")
print(f"Number of diagnosis instances: {len(diagnosis_instances)}")
print(f"Number of drug instances: {len(drug_instances)}")
print(f"Number of medical procedure instances: {len(medical_procedure_instances)}")
print(f"Number of drug prescription instances: {len(drug_prescription_instances)}")
print(f"Number of lab test event instances: {len(lab_test_event_instances)}")
print(f"Number of lab result instances: {len(lab_result_instances)}")
print(f"Number of substance instances: {len(substance_instances)}")


Number of patient instances: 398
Number of diagnosis instances: 795
Number of drug instances: 10953
Number of medical procedure instances: 644
Number of drug prescription instances: 10953
Number of lab test event instances: 20079
Number of lab result instances: 20079
Number of substance instances: 10953


In [11]:
# Print the first 10 instances of diagnosis_instances
for i, instance in enumerate(substance_instances):
    if i >= 10:
        break
    print("/".join(instance.split('/')[-4::2]))  # Print only the last part of the URI

3/12780018
6/12993725
13/16438060
17/14393409
17/12395381
8/13083369
11/19152594
17/16268354
47/14756995
6/18023872


In [ ]:
import re
import random
generated_qq = []
pattern = r"\{[a-zA-Z]+\}"
# Iterate through the question-query data and replace placeholders with actual instances
# Using counter to account for questions with multiple placeholders basically adding lots of templates with one placeholder less
# Random samplin used to reduce running time
counter = 0
while counter <len(question_query_data):
    entry = question_query_data[counter]
    counter += 1
    question = entry.get("Question")
    query = entry.get("Query")
    if "{SubjectPseudoIdentifier}" in entry["Question"] or "{SubjectPseudoIdentifier}" in entry["Query"]:
        for patient in patient_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = str(patient).split('/')[-1]
            updated_question = entry["Question"].replace("{SubjectPseudoIdentifier}", q_replace)
            updated_query = entry["Query"].replace("{SubjectPseudoIdentifier}", patient)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query})
        continue
    if "{Diagnosis}" in entry["Question"] or "{Diagnosis}" in entry["Query"]:
        for diagnosis in diagnosis_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = str(diagnosis).split('/')[-3]
            updated_question = entry["Question"].replace("{Diagnosis}", q_replace)
            updated_query = entry["Query"].replace("{Diagnosis}", diagnosis)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query})
        continue
    if "{Drug}" in entry["Question"] or "{Drug}" in entry["Query"]:
        sampled_drug_instances = random.sample(list(drug_instances), 10)
        for drug in sampled_drug_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = str(drug).split('/')[-1]
            updated_question = entry["Question"].replace("{Drug}", q_replace)
            updated_query = entry["Query"].replace("{Drug}", drug)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query})
        continue
    if "{MedicalProcedure}" in entry["Question"] or "{MedicalProcedure}" in entry["Query"]: 
        # Sample 10 medical procedure instances
        sampled_medical_procedure_instances = random.sample(list(medical_procedure_instances), 10)
        for procedure in sampled_medical_procedure_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = str(procedure).split('/')[-3]
            updated_question = entry["Question"].replace("{MedicalProcedure}", q_replace)
            updated_query = entry["Query"].replace("{MedicalProcedure}", procedure)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query})
        continue
    if "{DrugPrescription}" in entry["Question"] or "{DrugPrescription}" in entry["Query"]:
        # Sample 10 drug prescription instances
        sampled_drug_prescription_instances = random.sample(list(drug_prescription_instances), 10)

        for prescription in sampled_drug_prescription_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = str(prescription).split('/')[-1]
            updated_question = entry["Question"].replace("{DrugPrescription}", q_replace)
            updated_query = entry["Query"].replace("{DrugPrescription}", prescription)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query})
        continue
    if "{LabTestEvent}" in entry["Question"] or "{LabTestEvent}" in entry["Query"]:
        # Sample 10 lab test event instances
        sampled_lab_test_event_instances = random.sample(list(lab_test_event_instances), 10)

        for lab_test in sampled_lab_test_event_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = str(lab_test).split('/')[-3]
            updated_question = entry["Question"].replace("{LabTestEvent}", q_replace)
            updated_query = entry["Query"].replace("{LabTestEvent}", lab_test)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query})
        continue
    if "{LabResult}" in entry["Question"] or "{LabResult}" in entry["Query"]:
        # Sample 10 lab result instances
        sampled_lab_result_instances = random.sample(list(lab_result_instances), 10)

        for lab_result in sampled_lab_result_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = str(lab_result).split('/')[-1]
            updated_question = entry["Question"].replace("{LabResult}", q_replace)
            updated_query = entry["Query"].replace("{LabResult}", lab_result)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query})
        continue
    if "{Substance}" in entry["Question"] or "{Substance}" in entry["Query"]:
        # Sample 10 substance instances
        sampled_substance_instances = random.sample(list(substance_instances), 10)

        for substance in sampled_substance_instances:
            #Change this as per the URI to get the unique identifier for each class
            q_replace = "/".join(str(substance).split('/')[-4::2])
            updated_question = entry["Question"].replace("{Substance}", q_replace)
            updated_query = entry["Query"].replace("{Substance}", substance)
            if re.search(pattern, updated_question) is None and re.search(pattern, updated_query) is None:
                generated_qq.append({"Question": updated_question, "Query": updated_query})
            else:
                question_query_data.append({"Question": updated_question, "Query": updated_query})
        continue
len(generated_qq)

# Save the generated question-query pairs to a YAML file
output_yaml = "generated_question_query_pairs.yaml"
with open(output_yaml, mode="w", encoding="utf-8") as file:
    yaml.dump(generated_qq, file, default_flow_style=False, allow_unicode=True)

In [8]:
len(question_query_data)

250

In [26]:

# Define the pattern to match text surrounded by curly brackets
pattern = r"\{[a-zA-Z]+\}"

# Iterate through the question_query_data to find matches
for entry in generated_qq:
    question = entry.get("Question", "")
    query = entry.get("Query", "")
    
    # Check if the pattern exists in either the question or query
    if re.search(pattern, question) or re.search(pattern, query):
        print(f"Match found in entry:\nQuestion: {question}\nQuery: {query}\n")

In [141]:
filtered_question_query_data = [
    entry for entry in question_query_data
    if "{SubjectPseudoIdentifier}" in entry["Question"] or "{SubjectPseudoIdentifier}" in entry["Query"]
]

output_filtered_yaml = "filtered_question_query_data.yaml"

with open(output_filtered_yaml, mode="w", encoding="utf-8") as file:
    yaml.dump(filtered_question_query_data, file, default_flow_style=False, allow_unicode=True)

print(f"Filtered YAML file '{output_filtered_yaml}' created successfully.")

Filtered YAML file 'filtered_question_query_data.yaml' created successfully.


In [154]:
diagnosis_instances = set()

# Find all patient instances in the graph
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis'))):
    diagnosis_instances.add(str(s))
#print(patient_instances)
# Generate updated question-query pairs
# updated_question_query_data = []
diagnosis_qq = []
diagnosisquestion_query = []
for diagnosis in diagnosis_instances:
     for entry in filtered_question_query_data:
        q_replace = str(diagnosis).split('/')[-3:]
        q_replace = "/".join([q_replace[0], q_replace[2]])
        # print(q_replace)
        updated_question = entry["Question"].replace("{SubjectPseudoIdentifier}", q_replace)
        updated_query = entry["Query"].replace("{SubjectPseudoIdentifier}", patient)
        patient_qq.append({"Question": updated_question, "Query": updated_query})

# Print the updated data
print(len(patient_qq))
# output_filtered_yaml = "filtered_question_query_data.yaml"

# with open(output_filtered_yaml, mode="w", encoding="utf-8") as file:
#     yaml.dump(patient_qq, file, default_flow_style=False, allow_unicode=True)

# print(f"Filtered YAML file '{output_filtered_yaml}' created successfully.")

12728


# Creating CSV/YAML

In [52]:
import csv

output_csv = "question_query_pairs_100.csv"

data_to_write = []

# Adding labels to the data
data_to_write.extend([("Star 1 Hop", q, a) for q, a in qa_pairs])
data_to_write.extend([("Star 2 Hop", q, a) for q, a in new_qa_pairs])
data_to_write.extend([("Chain 2 Hop", q, a) for q, a in active_ingredient_qa_pairs])
data_to_write.extend([("Star 1 Hop", q, a) for q, a in procedure_qa_pairs])
data_to_write.extend([("Chain 3 Hop", q, a) for q, a in substance_code_qa_pairs])
data_to_write.extend([("Chain 1 Hop", q, a) for q, a in drug_qa_pairs])

print(len(data_to_write))

with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Label", "Question", "SPARQL Query"])  # Write the header
    writer.writerows(data_to_write)

print(f"CSV file '{output_csv}' created successfully.")

16587
CSV file 'question_query_pairs_100.csv' created successfully.


In [53]:
import yaml

output_yaml = "question_query_pairs_100.yaml"

# Consolidating data
data_to_write = {
    "Star 1 Hop": [{"Question": q, "SPARQL Query": a} for q, a in qa_pairs],
    "Star 2 Hop": [{"Question": q, "SPARQL Query": a} for q, a in new_qa_pairs],
    "Chain 2 Hop": [{"Question": q, "SPARQL Query": a} for q, a in active_ingredient_qa_pairs],
    "Star 1 Hop (Procedure)": [{"Question": q, "SPARQL Query": a} for q, a in procedure_qa_pairs],
    "Chain 3 Hop": [{"Question": q, "SPARQL Query": a} for q, a in substance_code_qa_pairs],
    "Chain 1 Hop": [{"Question": q, "SPARQL Query": a} for q, a in drug_qa_pairs],
}

with open(output_yaml, mode="w", encoding="utf-8") as file:
    yaml.dump(data_to_write, file, default_flow_style=False, allow_unicode=True)

print(f"YAML file '{output_yaml}' created successfully.")

YAML file 'question_query_pairs_100.yaml' created successfully.


# Complex Questions

## Question 5

In [55]:
# Generate question and SPARQL query pairs for the template:
# "Which patients, with the same lab results, have different diagnoses"

same_lab_diff_diag_qa_pairs = []

question = "Which patients, with the same lab test event, have different diagnoses?"
sparql_query = """
SELECT DISTINCT ?Patient1 ?Patient2 WHERE {
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?LabTestEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode1 .

    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?LabTestEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode2 .

    FILTER(?Patient1 != ?Patient2) .
    FILTER(?LabTestEvent1 = ?LabTestEvent2) .
    FILTER(?DiagnosisCode1 != ?DiagnosisCode2) .
}
"""

same_lab_diff_diag_qa_pairs.append((question, sparql_query.strip()))
print(same_lab_diff_diag_qa_pairs[0][0])
print(same_lab_diff_diag_qa_pairs[0][1])

Which patients, with the same lab test event, have different diagnoses?
SELECT DISTINCT ?Patient1 ?Patient2 WHERE {
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?LabTestEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode1 .

    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?LabTestEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent2 <https://www.biomedi

In [56]:
for row in g.query(same_lab_diff_diag_qa_pairs[0][1]):
    print([str(item) for item in row])

KeyboardInterrupt: 

## Question 6

In [ ]:

diff_diag_qa_pairs = []
question = "What active ingredients are prescribed to patients with different diagnoses from one another?"
sparql_query = """
SELECT DISTINCT ?ActiveIngredient WHERE {
    ?Patient1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier> .
    ?Patient2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier> .
    FILTER(?Patient1 != ?Patient2) .

    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode1 .

    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode2 .

    FILTER(?DiagnosisCode1 != ?DiagnosisCode2) .

    ?DrugPrescription1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DrugPrescription1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription> .
    ?DrugPrescription1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug1 .
    ?Drug1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient> ?ActiveIngredient .
}
"""

diff_diag_qa_pairs.append((question, sparql_query.strip()))
print(diff_diag_qa_pairs[0][0])
print(diff_diag_qa_pairs[0][1])



hello
What active ingredients are prescribed to patients with different diagnoses from one another?
SELECT DISTINCT ?ActiveIngredient WHERE {
    ?Patient1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier> .
    ?Patient2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier> .
    FILTER(?Patient1 != ?Patient2) .

    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode1 .

    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode2 .

    FILTER(?DiagnosisCode1 != ?DiagnosisCode2) .

    ?DrugPrescription1 <

In [ ]:
for row in g.query(diff_diag_qa_pairs[0][1]):
    print([str(item) for item in row])

## Question type 3

In [ ]:
# Generate question and SPARQL query pairs for the template:
# "Which patients, with the same lab results and the same diagnosis, have different Medical Procedures"

same_lab_diag_diff_proc_qa_pairs = []

# SPARQL query template
question = "Which patients, with the same lab results and the same diagnosis, have different Medical Procedures?"
sparql_query = """
SELECT DISTINCT ?Patient1 ?Patient2 WHERE {
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult1 .
    ?LabResult1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?ProcedureEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?ProcedureEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .
    
    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult2 .
    ?LabResult2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?ProcedureEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?ProcedureEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .
    
    FILTER(?Patient1 != ?Patient2) .
    FILTER(?LabResult1 = ?LabResult2) .
    FILTER(?Procedure1 != ?Procedure2) .
}
"""

same_lab_diag_diff_proc_qa_pairs.append((question, sparql_query.strip()))
print(same_lab_diag_diff_proc_qa_pairs[0][0])
print(same_lab_diag_diff_proc_qa_pairs[0][1])

Which patients, with the same lab results and the same diagnosis, have different Medical Procedures?
SELECT DISTINCT ?Patient1 ?Patient2 WHERE {
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult1 .
    ?LabResult1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?ProcedureEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?ProcedureEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .

    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult2 .


In [ ]:
for row in g.query(same_lab_diag_diff_proc_qa_pairs[0][1]):
    print([str(item) for item in row])

## Question 8

In [ ]:
patient_question_query_pairs = []

for s, p, o in g.triples((None, None, None)):
    if 'PATIENTS' in str(o):
        patient_id = str(o).split('/')[-1]
        #print(o)
        if patient_id.isdigit():
            question = f"What drugs, diagnosis, lab results and medical procedures did patient {patient_id} have?"
            sparql_query = f"""
            SELECT ?Drug ?Diagnosis ?LabResult ?MedicalProcedure WHERE {{
                ?DrugPres <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{o}> .
                ?DrugPres <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
                
                ?Diagnosis <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{o}> .
                
                ?LabResult <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{o}> .
                
                ?MedicalProcedure <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{o}> .
                
            }}
            """
            patient_question_query_pairs.append((question, sparql_query.strip()))

print(len(patient_question_query_pairs))
if patient_question_query_pairs:
    print(patient_question_query_pairs[0][0])
    print(patient_question_query_pairs[0][1])

64588
What drugs, diagnosis, lab results and medical procedures did patient 14346880 have?
SELECT ?Drug ?Diagnosis ?LabResult ?MedicalProcedure WHERE {
                ?DrugPres <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/14346880> .
                ?DrugPres <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .

                ?Diagnosis <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/14346880> .

                ?LabResult <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/14346880> .

                ?MedicalProcedure <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/14346880> .

            }


In [ ]:
for row in g.query(patient_question_query_pairs[0][1]):
    print([str(item) for item in row])

KeyboardInterrupt: 